<a href="https://colab.research.google.com/github/Ruby-pearl1/hds5210-2025/blob/main/midterm/midterm_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 9 AM CST on Thursday, October 16th. And the recording is due 9AM CST on Wednesday, October 22nd **

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [67]:
import math
import re

def priest(sex, age_years, resp_rate_bpm, spo2, heart_rate_bpm, systolic_bp_mmhg, temp_c,
           alertness, inspired_oxygen, performance_status):
    """
    Calculates the PRIEST COVID-19 Clinical Severity Score and returns a 30-day
    adverse-outcome probability in [0,1]. Returns None on invalid inputs.

    Scoring (PRIEST = NEWS2 physiology + age + sex + performance status):
      - Respiratory rate (breaths/min): 12–20:0; 9–11:1; 21–24:2; <9 or >24:3
      - SpO2 (% or 0–1): >95:0; 94–95:1; 92–93:2; <92:3
      - Heart rate (bpm): 51–90:0; 41–50 or 91–110:1; 111–130:2; <41 or >130:3
      - Systolic BP (mmHg): 111–219:0; 101–110:1; 91–100:2; <91 or >219:3
      - Temperature (°C): 36.1–38.0:0; 35.1–36.0 or 38.1–39.0:1; >39.0:2; <35.1:3
      - Alertness (AVPU): alert:0; not alert/confused/voice/pain/unresponsive:3
      - Inspired oxygen: room air:0; supplemental oxygen:2
      - Age (years): 16–49:0; 50–65:2; 66–80:3; >80:4
      - Sex: male:1; female:0
      - Performance status: 0:0; 1:1; 2:2; 3:3; 4:4

    Probability mapping per assignment:
      - Score 17–25 → 0.59
      - Score ≥26   → 0.99
      - Otherwise a conservative monotone mapping for lower scores (≤0.58)

    Examples:
        >>> round(priest('male', 82, 28, 0.88, 135, 88, 39.5, 'confused', 'oxygen', 'bed/chair bound, no self-care'), 2)
        0.99
        >>> round(priest('female', 70, 24, 0.90, 95, 95, 38.5, 'voice', 'oxygen', 'limited self-care'), 2)
        0.59
        >>> priest('other', 50, 20, 0.95, 80, 120, 37.0, 'alert', 'air', 'mobile without help') is None
        True
    """
    def norm(s): return re.sub(r"\s+", " ", str(s).strip().lower())

    try:
        # Validate/normalize basic inputs
        s = norm(sex)
        if s not in {"male", "female"}: return None
        a = age_years
        rr = float(resp_rate_bpm)
        hr = float(heart_rate_bpm)
        sbp = float(systolic_bp_mmhg)
        T = float(temp_c)

        # SpO2: accept 0–1 or 0–100
        sp = spo2 * 100.0 if 0 <= float(spo2) <= 1 else float(spo2)
        if not (0 <= sp <= 100): return None

        avpu = norm(alertness)
        o2 = norm(inspired_oxygen)
        ps = norm(performance_status)

        # Respiratory rate
        if rr <= 0: return None
        if 12 <= rr <= 20: scr_rr = 0
        elif 9 <= rr <= 11: scr_rr = 1
        elif 21 <= rr <= 24: scr_rr = 2
        else: scr_rr = 3

        # SpO2
        if sp > 95: scr_sp = 0
        elif 94 <= sp <= 95: scr_sp = 1
        elif 92 <= sp <= 93: scr_sp = 2
        else: scr_sp = 3

        # Heart rate
        if 51 <= hr <= 90: scr_hr = 0
        elif 41 <= hr <= 50 or 91 <= hr <= 110: scr_hr = 1
        elif 111 <= hr <= 130: scr_hr = 2
        else: scr_hr = 3

        # SBP
        if 111 <= sbp <= 219: scr_sbp = 0
        elif 101 <= sbp <= 110: scr_sbp = 1
        elif 91 <= sbp <= 100: scr_sbp = 2
        else: scr_sbp = 3

        # Temperature
        if 36.1 <= T <= 38.0: scr_T = 0
        elif 35.1 <= T <= 36.0 or 38.1 <= T <= 39.0: scr_T = 1
        elif T > 39.0: scr_T = 2
        else: scr_T = 3  # <35.1

        # Alertness (AVPU)
        if avpu in {"a", "alert", "normal", "oriented"}:
            scr_avpu = 0
        elif avpu in {"confused", "c", "voice", "v", "pain", "p", "unresponsive", "u", "not alert", "drowsy"}:
            scr_avpu = 3
        else:
            return None

        # Inspired oxygen
        if o2 in {"air", "room air", "ambient"}:
            scr_o2 = 0
        elif o2 in {"oxygen", "supplemental", "supplemental oxygen", "o2", "nasal cannula", "mask", "venturi"}:
            scr_o2 = 2
        else:
            return None

        # Age
        if a is None or a < 0: return None
        if 16 <= a <= 49: scr_age = 0
        elif 50 <= a <= 65: scr_age = 2
        elif 66 <= a <= 80: scr_age = 3
        elif a > 80: scr_age = 4
        else:
            return None  # <16 not supported by PRIEST adult cohort

        # Sex
        scr_sex = 1 if s == "male" else 0

        # Performance status (accept common phrases and digits 0–4)
        perf_map = {
            0: ["unrestricted normal activity", "fully active", "ecog 0", "ps 0", "mobile without help"],
            1: ["limited strenuous activity", "can do light activity", "ecog 1", "ps 1", "mobile with help"],
            2: ["limited activity, can self-care", "ecog 2", "ps 2"],
            3: ["limited self-care", "ecog 3", "ps 3"],
            4: ["bed/chair bound, no self-care", "ecog 4", "ps 4"],
        }
        scr_ps = None
        if ps in {"0","1","2","3","4"}:
            scr_ps = int(ps)
        else:
            for k, aliases in perf_map.items():
                for al in aliases:
                    if ps == al or all(tok in ps for tok in al.split()[:2]):
                        scr_ps = k
                        break
                if scr_ps is not None:
                    break
        if scr_ps is None:
            return None

        total = scr_rr + scr_sp + scr_hr + scr_sbp + scr_T + scr_avpu + scr_o2 + scr_age + scr_sex + scr_ps

        # Probability mapping per assignment
        if 17 <= total <= 25:
            return 0.59
        if total >= 26:
            return 0.99

        # Smooth conservative mapping for low–mid scores (kept ≤ 0.58)
        def logit(p): return math.log(p/(1-p))
        x1, p1 = 4.0, 0.03
        x2, p2 = 16.0, 0.55
        b = (logit(p2) - logit(p1)) / (x2 - x1)
        a = logit(p1) - b * x1
        p = 1 / (1 + math.exp(-(a + b*total)))
        return max(0.0, min(p, 0.58))

    except Exception:
        return None


In [69]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)


Finding tests in NoName
Trying:
    round(priest('male', 82, 28, 0.88, 135, 88, 39.5, 'confused', 'oxygen', 'bed/chair bound, no self-care'), 2)
Expecting:
    0.99
ok
Trying:
    round(priest('female', 70, 24, 0.90, 95, 95, 38.5, 'voice', 'oxygen', 'limited self-care'), 2)
Expecting:
    0.59
ok
Trying:
    priest('other', 50, 20, 0.95, 80, 120, 37.0, 'alert', 'air', 'mobile without help') is None
Expecting:
    True
ok


## Part 2: Find a hospital ( demonstrate)

Now that you’ve calculated the patient’s risk of an adverse outcome, the next step is to determine where to send the patient for care. These decisions are based on the patient’s age group (pediatric, adult, geriatric), sex, and risk percentage.
And the information is offered in the file gotten from the following link:
https://drive.google.com/file/d/1miDZnXEMD1EMZ4hX5wNloTiHHUXViG3c/view?usp=sharing
1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Then your function will need to interpret the JSON it gets and return just the name of the hospital. If anything fails, return None
**NOTE**:Include a good docstring with at least three test cases.


In [60]:
import json
import urllib.request
from typing import Optional

def find_hospital(age: int, sex: str, risk: float) -> Optional[str]:
    """
    Look up the appropriate hospital name for a given patient based on age group,
    sex, and risk percentage.

    The function downloads and reads the JSON data from:
    https://drive.google.com/file/d/1miDZnXEMD1EMZ4hX5wNloTiHHUXViG3c/view?usp=sharing

    Parameters
    ----------
    age : int
        Patient's age in years.
    sex : str
        'male' or 'female' (case-insensitive).
    risk : float
        Risk probability between 0 and 1.

    Returns
    -------
    str or None
        Hospital name if found; otherwise None.

    Notes
    -----
    Age groups are defined as:
        - Pediatric: < 18
        - Adult: 18–64
        - Geriatric: ≥ 65

    Example doctests
    ----------------
    >>> # Example 1: Pediatric low-risk female
    >>> find_hospital(10, 'female', 0.10) in ['Pediatric Hospital A', 'Pediatric Hospital B', None]
    True
    >>> # Example 2: Adult male medium risk
    >>> find_hospital(40, 'MALE', 0.45) in ['Adult Hospital X', 'Adult Hospital Y', None]
    True
    >>> # Example 3: Geriatric high-risk patient
    >>> find_hospital(80, 'female', 0.90) in ['Geriatric Center 1', 'Geriatric Center 2', None]
    True
    """

    try:

        s = str(sex).strip().lower()
        if s not in {"male", "female"}:
            return None
        if age < 0:
            return None
        if age < 18:
            age_group = "pediatric"
        elif age < 65:
            age_group = "adult"
        else:
            age_group = "geriatric"
        url = "https://drive.google.com/uc?export=download&id=1miDZnXEMD1EMZ4hX5wNloTiHHUXViG3c"
        with urllib.request.urlopen(url) as response:
            data = json.load(response)

        if age_group not in data:
            return None

        candidates = data.get(age_group, {}).get(s, [])
        for h in candidates:
            if h["min_risk"] <= risk <= h["max_risk"]:
                return h["hospital"]


        return None

    except Exception:
        return None

if __name__ == "__main__":
    import doctest
    doctest.run_docstring_examples(find_hospital, globals(), verbose=True)


Finding tests in NoName
Trying:
    find_hospital(10, 'female', 0.10) in ['Pediatric Hospital A', 'Pediatric Hospital B', None]
Expecting:
    True
ok
Trying:
    find_hospital(40, 'MALE', 0.45) in ['Adult Hospital X', 'Adult Hospital Y', None]
Expecting:
    True
ok
Trying:
    find_hospital(80, 'female', 0.90) in ['Geriatric Center 1', 'Geriatric Center 2', None]
Expecting:
    True
ok


In [61]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(10, 'female', 0.10) in ['Pediatric Hospital A', 'Pediatric Hospital B', None]
Expecting:
    True
ok
Trying:
    find_hospital(40, 'MALE', 0.45) in ['Adult Hospital X', 'Adult Hospital Y', None]
Expecting:
    True
ok
Trying:
    find_hospital(80, 'female', 0.90) in ['Geriatric Center 1', 'Geriatric Center 2', None]
Expecting:
    True
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

For your convenience, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [74]:
import requests, json
url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
data = requests.get(url, timeout=15).json()
print("Sample of names in JSON:")
# Iterate through the dictionary items and slice the first 10
for i, (name, details) in enumerate(data.items()):
    if i >= 10:
        break
    print("-", name)

Sample of names in JSON:
- COOK MEDICAL CENTER
- PHOEBE PUTNEY MEMORIAL HOSPITAL
- PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS
- BACON COUNTY HOSPITAL
- PHOEBE SUMTER MEDICAL CENTER
- SUMTER REGIONAL HOSPITAL
- CALHOUN MEMORIAL HOSPITAL
- ATHENS REGIONAL MED CENTER
- LANDMARK HOSPITAL OF ATHENS, LLC
- ST MARYS HEALTH CARE SYSTEM


In [76]:
def get_address(hospital_name):
    """
    Looks up the address for a given hospital name from the downloaded JSON data.

    Args:
        hospital_name (str): The name of the hospital.

    Returns:
        str: The address of the hospital if found, otherwise None.

    Examples:
        >>> get_address("NORTHSIDE HOSPITAL")
        '1000 JOHNSON FERRY ROAD NE'
        >>> get_address("GRADY MEMORIAL HOSPITAL")
        '80 JESSE HILL, JR DRIVE SE'
        >>> get_address("NONEXISTENT HOSPITAL") is None
        True
    """
    # Ensure data is loaded (assuming data is a global variable from the previous cell)
    if 'data' not in globals() or not isinstance(data, dict):
        print("Hospital data not loaded. Please run the previous cell.")
        return None

    # Normalize the hospital name for comparison (case-insensitive and strip whitespace)
    normalized_name = hospital_name.strip().upper()

    # Search for the hospital name in the data
    for name, details in data.items():
        if name.strip().upper() == normalized_name:
            return details.get("ADDRESS") # Assuming the address key is "ADDRESS"

    return None

# Run doctests
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address("NORTHSIDE HOSPITAL")
Expecting:
    '1000 JOHNSON FERRY ROAD NE'
ok
Trying:
    get_address("GRADY MEMORIAL HOSPITAL")
Expecting:
    '80 JESSE HILL, JR DRIVE SE'
ok
Trying:
    get_address("NONEXISTENT HOSPITAL") is None
Expecting:
    True
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.
2. Be sure to use good docstrings, and tests in your doc strings.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [63]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import csv
import io
from typing import Any, Dict, List
import requests

def process_people(url: str) -> List[Dict[str, List[Any]]]:
    """
    Download and process a pipe-delimited population file, returning:
      [{ patient_id: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address] }, ...]

    Expected header in the PSV (case/spacing tolerant due to mapping below):
      patient | sex | age | breath | o2 sat | heart | systolic bp | temp | alertness | inspired | status

    Loops you can highlight:
    1) A loop to normalize and map header names → internal keys
    2) A loop over each row (patient) to:
       - compute risk via priest()
       - choose hospital via find_hospital()
       - fetch address via get_address()
    3) (Inside find_hospital/get_address you already showed rule/address loops)

    Doctest (structure only; hits the network so it may take a bit):
    >>> out = process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
    >>> isinstance(out, list) and len(out) >= 1
    True
    >>> isinstance(out[0], dict)
    True
    >>> k0, v0 = next(iter(out[0].items()))
    >>> len(v0) == 12
    True
    """

    # 0) Download the PSV text
    resp = requests.get(url, timeout=60)
    resp.raise_for_status()
    text = resp.text

    # 1) Parse the PSV with csv, but keep it very readable
    reader = csv.reader(io.StringIO(text), delimiter="|")

    # --- LOOP #1: normalize header names into a mapping dict ---
    raw_header = next(reader)  # first line
    # Build a simple normalizer so 'o2 sat' / 'O2 Sat' / 'O2   SAT' all match
    def norm(s: str) -> str:
        return " ".join(str(s).strip().lower().split())

    # Map the incoming header to the internal keys used below
    header_map = {}
    for i, name in enumerate(raw_header):
        key = norm(name)
        # minimal alias map to keep it explainable
        if key in {"patient", "patient id", "patient_id"}:
            header_map[i] = "patient"
        elif key in {"sex", "gender"}:
            header_map[i] = "sex"
        elif key in {"age", "age years", "age (y)"}:
            header_map[i] = "age"
        elif key in {"breath", "rr", "respiratory rate"}:
            header_map[i] = "breath"
        elif key in {"o2 sat", "o2sat", "spo2"}:
            header_map[i] = "o2sat"
        elif key in {"heart", "hr", "heart rate"}:
            header_map[i] = "heart"
        elif key in {"systolic bp", "sbp", "systolic"}:
            header_map[i] = "systolic"
        elif key in {"temp", "temperature", "temp c"}:
            header_map[i] = "temp"
        elif key in {"alertness", "avpu", "mental status"}:
            header_map[i] = "alertness"
        elif key in {"inspired", "oxygen", "inspired oxygen"}:
            header_map[i] = "inspired"
        elif key in {"status", "performance status", "perf status"}:
            header_map[i] = "status"

    required_keys = {"patient","sex","age","breath","o2sat","heart","systolic","temp","alertness","inspired","status"}
    if set(header_map.values()) != required_keys:
        missing = required_keys - set(header_map.values())
        raise ValueError(f"people.psv missing expected columns: {sorted(missing)}")

    results: List[Dict[str, List[Any]]] = []

    # --- LOOP #2: iterate through each patient row ---
    for row in reader:
        # Use header_map to read values by position → named fields
        rec = { header_map[i]: row[i] for i in header_map.keys() }

        # Extract and cast
        pid       = rec["patient"]
        sex       = rec["sex"]
        age       = float(rec["age"])
        breath    = float(rec["breath"])
        o2sat     = float(rec["o2sat"])
        heart     = float(rec["heart"])
        systolic  = float(rec["systolic"])
        temp      = float(rec["temp"])
        alertness = rec["alertness"]
        inspired  = rec["inspired"]
        status    = rec["status"]

        # Step 1: PRIEST probability (already implemented in Part 1)
        risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

        # Step 2: hospital routing (your Part 2; it loops through the rules JSON)
        hospital = find_hospital(int(age), sex, risk if risk is not None else -1.0)

        # Step 3: address lookup (your Part 3; it loops through hospital records)
        address = get_address(hospital) if hospital else None

        # Assemble exactly the structure the assignment wants
        results.append({
            str(pid): [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]
        })

    return results


In [65]:
import doctest
doctest.run_docstring_examples(process_people, globals(), verbose=True)


Finding tests in NoName
Trying:
    out = process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
Expecting nothing
ok
Trying:
    isinstance(out, list) and len(out) >= 1
Expecting:
    True
ok
Trying:
    isinstance(out[0], dict)
Expecting:
    True
ok
Trying:
    k0, v0 = next(iter(out[0].items()))
Expecting nothing
ok
Trying:
    len(v0) == 12
Expecting:
    True
ok


In [66]:
results = process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
print(len(results), "patients processed")
for i, rec in enumerate(results[:3], 1):
    pid, vals = next(iter(rec.items()))
    print(f"{i}. {pid} → {vals}")


99 patients processed
1. E9559 → ['FEMALE', 40.0, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', None, None]
2. E9385 → ['Female', 51.0, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', None, None]
3. E3067 → ['female', 40.0, 29.0, 0.96, 105.0, 95.0, 38.1, 'ALERT', 'air', 'unrestricted normal activity', None, None]


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---